# Fast F1 - Save data

## Import libraries

In [3]:
import fastf1
import matplotlib.pyplot as plt
import pandas as pd

from datetime import datetime

In [4]:
def get_schedule_until_now(year: int=2024):
    '''
        Filters schedule of races until today
    '''

    # Retrieve the event schedule for the specified season
    schedule = fastf1.get_event_schedule(year)

    # Filter sessions until current date
    schedule = schedule[schedule['EventDate'] <= datetime.today()]

    # Do not need the preseason results
    schedule = schedule[schedule['EventName'] != 'Pre-Season Testing']

    return schedule

In [5]:
schedule = get_schedule_until_now(2024)


req         WARNING 	DEFAULT CACHE ENABLED! (1.9 GB) /home/michail/.cache/fastf1


In [ ]:
schedule[]

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,...,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport
1,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,2024-03-02,Bahrain Grand Prix,conventional,Practice 1,2024-02-29 14:30:00+03:00,2024-02-29 11:30:00,...,Practice 3,2024-03-01 15:30:00+03:00,2024-03-01 12:30:00,Qualifying,2024-03-01 19:00:00+03:00,2024-03-01 16:00:00,Race,2024-03-02 18:00:00+03:00,2024-03-02 15:00:00,True
2,2,Saudi Arabia,Jeddah,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2024,2024-03-09,Saudi Arabian Grand Prix,conventional,Practice 1,2024-03-07 16:30:00+03:00,2024-03-07 13:30:00,...,Practice 3,2024-03-08 16:30:00+03:00,2024-03-08 13:30:00,Qualifying,2024-03-08 20:00:00+03:00,2024-03-08 17:00:00,Race,2024-03-09 20:00:00+03:00,2024-03-09 17:00:00,True
3,3,Australia,Melbourne,FORMULA 1 ROLEX AUSTRALIAN GRAND PRIX 2024,2024-03-24,Australian Grand Prix,conventional,Practice 1,2024-03-22 12:30:00+11:00,2024-03-22 01:30:00,...,Practice 3,2024-03-23 12:30:00+11:00,2024-03-23 01:30:00,Qualifying,2024-03-23 16:00:00+11:00,2024-03-23 05:00:00,Race,2024-03-24 15:00:00+11:00,2024-03-24 04:00:00,True
4,4,Japan,Suzuka,FORMULA 1 MSC CRUISES JAPANESE GRAND PRIX 2024,2024-04-07,Japanese Grand Prix,conventional,Practice 1,2024-04-05 11:30:00+09:00,2024-04-05 02:30:00,...,Practice 3,2024-04-06 11:30:00+09:00,2024-04-06 02:30:00,Qualifying,2024-04-06 15:00:00+09:00,2024-04-06 06:00:00,Race,2024-04-07 14:00:00+09:00,2024-04-07 05:00:00,True
5,5,China,Shanghai,FORMULA 1 LENOVO CHINESE GRAND PRIX 2024,2024-04-21,Chinese Grand Prix,sprint_qualifying,Practice 1,2024-04-19 11:30:00+08:00,2024-04-19 03:30:00,...,Sprint,2024-04-20 11:00:00+08:00,2024-04-20 03:00:00,Qualifying,2024-04-20 15:00:00+08:00,2024-04-20 07:00:00,Race,2024-04-21 15:00:00+08:00,2024-04-21 07:00:00,True
6,6,United States,Miami,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2024,2024-05-05,Miami Grand Prix,sprint_qualifying,Practice 1,2024-05-03 12:30:00-04:00,2024-05-03 16:30:00,...,Sprint,2024-05-04 12:00:00-04:00,2024-05-04 16:00:00,Qualifying,2024-05-04 16:00:00-04:00,2024-05-04 20:00:00,Race,2024-05-05 16:00:00-04:00,2024-05-05 20:00:00,True
7,7,Italy,Imola,FORMULA 1 MSC CRUISES GRAN PREMIO DEL MADE IN ...,2024-05-19,Emilia Romagna Grand Prix,conventional,Practice 1,2024-05-17 13:30:00+02:00,2024-05-17 11:30:00,...,Practice 3,2024-05-18 12:30:00+02:00,2024-05-18 10:30:00,Qualifying,2024-05-18 16:00:00+02:00,2024-05-18 14:00:00,Race,2024-05-19 15:00:00+02:00,2024-05-19 13:00:00,True
8,8,Monaco,Monaco,FORMULA 1 GRAND PRIX DE MONACO 2024,2024-05-26,Monaco Grand Prix,conventional,Practice 1,2024-05-24 13:30:00+02:00,2024-05-24 11:30:00,...,Practice 3,2024-05-25 12:30:00+02:00,2024-05-25 10:30:00,Qualifying,2024-05-25 16:00:00+02:00,2024-05-25 14:00:00,Race,2024-05-26 15:00:00+02:00,2024-05-26 13:00:00,True
9,9,Canada,Montréal,FORMULA 1 AWS GRAND PRIX DU CANADA 2024,2024-06-09,Canadian Grand Prix,conventional,Practice 1,2024-06-07 13:30:00-04:00,2024-06-07 17:30:00,...,Practice 3,2024-06-08 12:30:00-04:00,2024-06-08 16:30:00,Qualifying,2024-06-08 16:00:00-04:00,2024-06-08 20:00:00,Race,2024-06-09 14:00:00-04:00,2024-06-09 18:00:00,True
10,10,Spain,Barcelona,FORMULA 1 ARAMCO GRAN PREMIO DE ESPAÑA 2024,2024-06-23,Spanish Grand Prix,conventional,Practice 1,2024-06-21 13:30:00+02:00,2024-06-21 11:30:00,...,Practice 3,2024-06-22 12:30:00+02:00,2024-06-22 10:30:00,Qualifying,2024-06-22 16:00:00+02:00,2024-06-22 14:00:00,Race,2024-06-23 15:00:00+02:00,2024-06-23 13:00:00,True


In [ ]:
## TODO
## - Maybe reorganise the columns when saving or better leave for later preprocessing? 
##   But might be unneccessary disk space if I won't use the other columns.

year = 2024

schedule = get_schedule_until_now(year)

# Create an empty list to collect all laps data
all_laps = []

# Create an empty list to collect all race results data
all_race_results = []

# Create an empty list to collect all qualifying results data
all_quali_results = []

# Iterate over each event in the schedule
for _, event in schedule.iterrows():
    event_name = event['EventName']

    # Load the race session
    race_session = fastf1.get_session(year, event_name, 'R')
    race_session.load()  # Load the session data
    
    # Add the event name as a new column in the laps DataFrame
    laps = race_session.laps
    laps['EventName'] = event_name
    
    # Append the modified laps data to the list
    all_laps.append(laps)
    print(f"Laps data loaded for {event_name}")

    # Add the event name as a new column in the results DataFrame
    race_results = race_session.results
    race_results['EventName'] = event_name

    # Append the modified race results data to the list
    all_race_results.append(race_results)
    print(f"Race results data loaded for {event_name}")

    # Load the qualifying session
    quali_session = fastf1.get_session(year, event_name, 'Q')
    quali_session.load()
    
    # Add qualifying results data
    quali_results = quali_session.results
    quali_results['EventName'] = event_name

    # Append the modified qualifying results data to the list
    all_quali_results.append(quali_results)
    print(f"Qualifying results data loaded for {event_name}")

# Concatenate all collected laps data into a single DataFrame
if all_laps:
    all_laps_df = pd.concat(all_laps, ignore_index=True)
    
    # Save the DataFrame to a CSV file
    output_file = f'data/gps_{year}_season_laps.csv'
    all_laps_df.to_csv(output_file, index=False)
    print(f"Laps data for {year} saved to {output_file}")
else:
    print("No laps data available for the selected season.")

# Save race results data to a CSV file
if all_race_results:
    all_race_results_df = pd.concat(all_race_results, ignore_index=True)
    race_results_output_file = f'data/gps_{year}_season_results.csv'
    all_race_results_df.to_csv(race_results_output_file, index=False)
    print(f"Race results data for {year} saved to {race_results_output_file}")
else:
    print("No race results data available for the selected season.")

# Save qualifying results data to a CSV file
if all_quali_results:
    all_quali_results_df = pd.concat(all_quali_results, ignore_index=True)
    quali_results_output_file = f'data/gps_{year}_season_quali.csv'
    all_quali_results_df.to_csv(quali_results_output_file, index=False)
    print(f"Qualifying results data for {year} saved to {quali_results_output_file}")
else:
    print("No qualifying results data available for the selected season.")


NameError: name 'year' is not defined